In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

In [6]:
df=pd.read_csv(r"C:\Users\ADMIN\Desktop\Hoan\Portfolio\job_scraping\data\raw_data.csv")
df.head()

,source,job,url_job,job_id,posting_date,job_title,salary,location,exp,level,industry,company_name,description
0,Jobsgo,Data Analyst,https://jobsgo.vn/viec-lam/nhan-vien-phan-tich...,1765776,19-11-2025,Nhân Viên Phân Tích Dữ Liệu Và Xử Lý Đơn Hàng,Từ 15 triệu VNĐ,Hà Nội,Trên 1 năm,Nhân Viên/Chuyên Viên,Kinh doanh,Công Ty Cổ Phần Tân Phú Việt Nam,Chi Tiết Công Việc\nMô tả công việc:\n1. Xử lý...
1,Jobsgo,Data Analyst,https://jobsgo.vn/viec-lam/business-analyst-mi...,1757206,19-11-2025,Business Analyst (Middle),Đến 25 triệu VNĐ,Hà Nội,Trên 3 năm,Nhân Viên/Chuyên Viên,"Kinh Doanh,Công nghệ thông tin",Công Ty Cổ Phần Công Nghệ Phần Mềm Tinh Vân,Chi Tiết Công Việc\nMô tả công việc:\nTham gia...
2,Jobsgo,Data Analyst,https://jobsgo.vn/viec-lam/data-analyst-collec...,1764613,19-11-2025,"Data Analyst Collection Tại Vp Bình Thạnh , H...",Thỏa thuận,Hồ Chí Minh,3 - 5 năm,Nhân Viên/Chuyên Viên,Kinh doanh,Công Ty Tài chính TNHH HD SAISON,Chi Tiết Công Việc\nMô tả công việc:\nVận hành...
3,Jobsgo,Data Analyst,https://jobsgo.vn/viec-lam/sales-assistant-tie...,1767909,19-11-2025,Sales Assistant (Tiếng Nhật N1) (27037) [Phườn...,30 - 60 triệu VNĐ,Hà Nội,3 - 5 năm,Nhân Viên/Chuyên Viên,Tư vấn,Công Ty TNHH Reeracoen Việt Nam,Chi Tiết Công Việc\nMô tả công việc:\n- Là trợ...
4,Jobsgo,Data Analyst,https://jobsgo.vn/viec-lam/giam-sat-cuoc-goi-t...,1768668,19-11-2025,Giám Sát Cuộc Gọi (Trình Độ Tiếng Nhật N2) (27...,14 - 20 triệu VNĐ,Hà Nội,1 - 2 năm,Nhân Viên/Chuyên Viên,Tư vấn,Công Ty TNHH Reeracoen Việt Nam,Chi Tiết Công Việc\nMô tả công việc:\n- Là trợ...


In [ ]:
def process_location(location):
    if pd.isna(location):
        return None

    # Nếu có "&" thì lấy phần trước "&", còn lại split theo dấu ","
    if "&" in location:
        return location.split("&")[0].strip()
    elif "," in location:
        return location.split(",")[0].strip()  
    
df['location'].apply(process_location)

0           Hà Nội
1           Hà Nội
2      Hồ Chí Minh
3           Hà Nội
4           Hà Nội
          ...     
775    Hồ Chí Minh
776         Hà Nội
777         Hà Nội
778         Hà Nội
779         Hà Nội
Name: location, Length: 780, dtype: object

In [50]:
# Xử lý date
test=df.copy()
test['month']=pd.to_datetime(df['posting_date'], dayfirst=True).dt.month
test['year']=pd.to_datetime(df['posting_date'], dayfirst=True).dt.year
test[['posting_date', 'month', 'year']].head()

,posting_date,month,year
0,18-11-2025,11,2025
1,18-11-2025,11,2025
2,18-11-2025,11,2025
3,18-11-2025,11,2025
4,18-11-2025,11,2025


In [51]:
test.columns

Index(['source', 'job', 'url_job', 'job_id', 'posting_date', 'job_title',
       'salary', 'location', 'exp', 'level', 'industry', 'company_name',
       'description', 'month', 'year'],
      dtype='object')

In [52]:
df['salary'].unique()

array(['20 - 30 triệu VNĐ', 'Thỏa thuận', 'Từ 20 triệu VNĐ', 'Thoả thuận',
       'Tới 20 triệu', 'Tới 13 triệu', '10 - 20 triệu', '20 - 30 triệu',
       '10 - 25 triệu', '20 - 26 triệu', '25 - 40 triệu', 'Tới 60 triệu',
       '30 - 45 triệu', 'Tới 18 triệu', '16 - 20 triệu', '15 - 25 triệu',
       '25 - 35 triệu', '3 - 10 triệu', '15 - 30 triệu', '25 - 28 triệu'],
      dtype=object)

In [53]:
# Xử lý salary
# idea: chỉ giữ lại các ký tự số (dấu "." cho float) và dấu "-" dùng để tách khoảng
def process_salary(salary):
    if pd.isna(salary):
        return None
    
    salary_str=unidecode(salary.lower().strip()) # sử dụng unidecode để loại bỏ unicode
    # Kiểm tra có cụm từ "thỏa thuận" hoặc "thương lượng"
    if salary_str in ["thoa thuan", "thuong luong"]:
        return 0
    
    pattern=r'\s*[^\d.-]' # loại bỏ toàn bộ ký tự không phải số, dấu "." và "-" (bao gồm cả khoảng trắng)
    num_salary=re.sub(pattern, "", salary_str)

    # Xử lý range
    if "-" in num_salary:
        x=float(num_salary.split("-")[0])
        y=float(num_salary.split("-")[1])
        avg_salary=(x+y)/2
    else:
        avg_salary=float(num_salary)

    if "trieu" in salary_str:
        return avg_salary
    elif "usd" in salary_str:
        return avg_salary*0.026 # giả sử 1 USD = 26000 VNĐ
    
test=df['salary'].apply(process_salary)
test[:5]

0    25.0
1     0.0
2     0.0
3     0.0
4    20.0
Name: salary, dtype: float64

In [54]:
df['location'].unique()

array(['Hồ Chí Minh', 'Hà Nội', 'Hà Nội, Bắc Từ Liêm', 'Hà Nội, Cầu Giấy',
       'Hà Nội, Phường Nghĩa Đô', 'Hà Nội & 3 nơi khác',
       'Hà Nội, Phường Hà Đông', 'Hà Nội, Hà Đông',
       'Hà Nội, Phường Yên Hòa', 'Hà Nội, Phường Cầu Giấy',
       'Hà Nội, Phường Bạch Mai', 'Hà Nội, Phường Thanh Xuân',
       'Hà Nội, Hoàn Kiếm', 'Hà Nội, Đống Đa', 'Hà Nội, Thanh Xuân',
       'Hà Nội, Nam Từ Liêm', 'Đà Nẵng, Ngũ Hành Sơn',
       'Đà Nẵng, Bình Định', 'Hà Nội, Hai Bà Trưng',
       'Hà Nội, Phường Vĩnh Tuy', 'Hồ Chí Minh, Tân Bình',
       'Hồ Chí Minh, Quận 1', 'Hồ Chí Minh, Thủ Đức', 'Hà Nội, Ba Đình',
       'Hồ Chí Minh, Quận 2', 'Hồ Chí Minh, Quận 9',
       'Bình Dương & 3 nơi khác', 'Hồ Chí Minh, Quận 7', 'Hưng Yên',
       'Đà Nẵng, Cẩm Lệ', 'Hà Nội, Phường Đống Đa',
       'Hồ Chí Minh, Phường Phú Nhuận', 'Hồ Chí Minh, Phường Tân Thuận',
       'Hồ Chí Minh, Quận 10', 'Bà Rịa-Vũng Tàu, TP Vũng Tàu'],
      dtype=object)

In [55]:
# Xử lý location
def process_location(location):
    if pd.isna(location):
        return None
    
    # Nếu có "&" thì lấy phần trước "&", còn lại split theo dấu ","
    if "&" in location:
        return location.split("&")[0]
    else:
        return location.split(",")
    
test=df[df['location'].str.contains(",") | df['location'].str.contains("&")]['location'].apply(process_location)
test[:5]

7         [Hà Nội,  Bắc Từ Liêm]
8            [Hà Nội,  Cầu Giấy]
9     [Hà Nội,  Phường Nghĩa Đô]
10                       Hà Nội 
11     [Hà Nội,  Phường Hà Đông]
Name: location, dtype: object

In [56]:
df['exp'].unique()

array(['3 - 5 năm', 'Không yêu cầu', 'Trên 2 năm', '2 năm', '5 năm',
       '1 năm', 'Dưới 1 năm', '3 năm', '4 năm', 'Trên 5 năm'],
      dtype=object)

In [57]:
# Xử lý exp
# idea: chỉ giữ lại các ký tự số (dấu "." cho float) và dấu "-" dùng để tách khoảng
def process_exp(exp):
    if pd.isna(exp):
        return None
    
    exp_str=unidecode(exp.lower().strip()) # sử dụng unidecode để loại bỏ unicode
    # Kiểm tra có cụm từ "không yêu cầu"
    if exp_str == "khong yeu cau":
        return 0
    
    pattern=r'\s*[^\d.-]' # loại bỏ toàn bộ ký tự không phải số, dấu "." và "-" (bao gồm cả khoảng trắng)
    num_exp=re.sub(pattern, "", exp_str)

    # Xử lý range
    if "-" in num_exp:
        x=float(num_exp.split("-")[0])
        y=float(num_exp.split("-")[1])
        return (x+y)/2
    else:
        return num_exp

test=df['exp'].apply(process_exp)
test[:5]

0    4.0
1      0
2      0
3      0
4      2
Name: exp, dtype: object

In [58]:
df['level'].unique()

array(["['Nhân Viên/Chuyên Viên']", "['Trưởng Nhóm/Trưởng Phòng']",
       "['Nhân viên']", nan, "['Trưởng nhóm']", "['Trưởng/Phó phòng']",
       "['Quản lý / Giám sát']", "['Thực tập sinh']"], dtype=object)

In [59]:
# Xử lý level
def process_level(level):
    if pd.isna(level):
        return None
    
    # xóa khoảng trắng trước sau dấu "/" và chuyển thành dấu "," rồi tách bằng dấu ","
    pattern=r'\s*/\s*'
    new_level=re.sub(pattern, '/', level).replace("/", ",").strip("[]").split(",")
    return new_level

df['level'].apply(process_level)

0     ['Nhân Viên, Chuyên Viên']
1     ['Nhân Viên, Chuyên Viên']
2     ['Nhân Viên, Chuyên Viên']
3     ['Nhân Viên, Chuyên Viên']
4     ['Nhân Viên, Chuyên Viên']
                 ...            
74                          None
75                 ['Nhân viên']
76                 ['Nhân viên']
77                 ['Nhân viên']
78                 ['Nhân viên']
Name: level, Length: 79, dtype: object

In [60]:
df['industry'].unique()

array(["['Kinh doanh, Sản xuất/Vận hành sản xuất']", "['Nhà hàng']",
       "['Kinh doanh']", nan, "['IT - Phần mềm']", "['Viễn thông']",
       "['Giáo dục / Đào tạo']",
       "['Dược phẩm / Y tế / Công nghệ sinh học']", "['Ngân hàng']",
       "['Tài chính']", "['Giải trí']", "['Khác']",
       "['Điện tử / Điện lạnh']", "['Thời trang']", "['Sản xuất']"],
      dtype=object)

In [61]:
def process_industry(industry):
    if pd.isna(industry):
        return None
    
    # xóa khoảng trắng trước sau dấu "/" và dấu ",", chuyển toàn bộ thành dấu "," rồi tách bằng dấu ","
    pattern=r'\s*[/,]\s*'
    new_industry=re.sub(pattern, '/', industry).replace("/", ",").strip("[]").split(",")
    return new_industry

df['industry'].apply(process_industry)

0     ['Kinh doanh, Sản xuất, Vận hành sản xuất']
1                                    ['Nhà hàng']
2                                  ['Kinh doanh']
3                                  ['Kinh doanh']
4                                  ['Kinh doanh']
                         ...                     
74                                           None
75                                   ['Sản xuất']
76                                  ['Ngân hàng']
77                                  ['Tài chính']
78                          ['Giáo dục, Đào tạo']
Name: industry, Length: 79, dtype: object

In [62]:
keywords=[
    # Tools for DA
    'Python', 'R', 'SQL', 'Tableau', 'Power BI', 'Qlik', 'Looker', 'Data Studio',
    'Mysql', 'Postgresql', 'Oracle', 'SQL Server', 'Excel', 'Google Sheet', 'Powerpoint', 'SPSS',
    # Tools for DE
    'Mongodb', 'Bigquery',
    'Google Sheet', 'Spark', 'Powerpoint', 'SPSS',
    'Amazon S3', 'Google Cloud Storage', 'Azure', 'Kafka', 'Flink', 'Hive', 'Prestom',
    'Airflow', 'Perfect', 'Luigi', 'Alation', 'Collibra', 'Redshift', 'Snowflake', 'Docker',
    'Kubernetes', 'Terraform', 'Pulumi', 'Informatica', 'Talend', 'SSIS', 'ODI',
    # Tools for DS, ML
    "Matplotlib", "Spark", "Hadoop", "TensorFlow", "SAS", "BigML", "Scikit-learn", "Knime", "Matlab", "Pytorch", "Cloud Computing", "Keras", "Rapid Miner",
    "Azure Machine Learning", "Watson Studio", "Mahout", "RapidMiner", "Shogun", "OpenNN", "SageMaker",
    # Tools for ML
    "Visio", "Jira", "Excel", "Power BI", "LucidChart", "Balsamiq", "Clickup", "Enterprise Architect", "Wrike", "Blueprint", "Confluence", "NetSuite", "Niffy",
    "Trello"
]

def process_description(description, keywords):
    if pd.isna(description):
        return []
    
    # Chuyển văn bản về chữ thường, loại bỏ space
    pattern=r'\s*'
    new_desc = re.sub(pattern, '', str(description).lower())
    keyword_map = {}
    for kw in pd.unique(keywords):
        if not kw:
            continue
        norm_kw = re.sub(r'\s*', '', kw.lower())
        keyword_map[norm_kw] = kw

    matched_original = []
    for norm_kw, original_kw in keyword_map.items():
        if norm_kw in new_desc:
            matched_original.append(original_kw)
    
    return matched_original

df['description'].apply(lambda x: process_description(x, keywords))

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6148\116232496.py:27: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  for kw in pd.unique(keywords):


0     [Python, R, SQL, Power BI, Mysql, Oracle, SQL ...
1     [Python, R, SQL, Looker, Mysql, Postgresql, Mo...
2                [Python, R, SQL, Spark, Azure, Hadoop]
3                                                   [R]
4     [Python, R, SQL, Power BI, Excel, TensorFlow, ...
                            ...                        
74                                                   []
75                     [Python, R, TensorFlow, Pytorch]
76                [Python, R, Matplotlib, Scikit-learn]
77            [Python, R, Excel, Scikit-learn, Pytorch]
78                                          [Python, R]
Name: description, Length: 79, dtype: object

In [6]:
df=pd.read_csv(r"C:\Users\ADMIN\Desktop\Hoan\Portfolio\job_scraping\data\processed_data.csv")
df.head()

,source,job,month,year,job_id,job_title,salary,location,exp,level,industry,company_name,description
0,Jobsgo,Data Analyst,11,2025,1765776,Nhân Viên Phân Tích Dữ Liệu Và Xử Lý Đơn Hàng,15.0,Hà Nội,1.0,"Nhân Viên,Chuyên Viên",Kinh doanh,Công Ty Cổ Phần Tân Phú Việt Nam,"Python,R,Power BI"
1,Jobsgo,Data Analyst,11,2025,1757206,Business Analyst (Middle),25.0,Hà Nội,3.0,"Nhân Viên,Chuyên Viên","Kinh Doanh,Công nghệ thông tin",Công Ty Cổ Phần Công Nghệ Phần Mềm Tinh Vân,"R,ODI,Visio"
2,Jobsgo,Data Analyst,11,2025,1764613,"Data Analyst Collection Tại Vp Bình Thạnh , H...",0.0,Hồ Chí Minh,4.0,"Nhân Viên,Chuyên Viên",Kinh doanh,Công Ty Tài chính TNHH HD SAISON,"Python,R,SQL,Tableau,Power BI,Mysql,Postgresql..."
3,Jobsgo,Data Analyst,11,2025,1767909,Sales Assistant (Tiếng Nhật N1) (27037) [Phườn...,45.0,Hà Nội,4.0,"Nhân Viên,Chuyên Viên",Tư vấn,Công Ty TNHH Reeracoen Việt Nam,"R,Excel"
4,Jobsgo,Data Analyst,11,2025,1768668,Giám Sát Cuộc Gọi (Trình Độ Tiếng Nhật N2) (27...,17.0,Hà Nội,1.5,"Nhân Viên,Chuyên Viên",Tư vấn,Công Ty TNHH Reeracoen Việt Nam,R


In [13]:
def explode_data(df):
    cols_to_explode = ['location', 'level', 'industry', 'description']

    new_df=df.copy()
    for col in cols_to_explode:
        new_df[col] = df[col].astype(str).str.split(',') \
            .apply(lambda x: [item.strip() for item in x if item.strip() and item.strip() != 'nan'])

    max_lengths = new_df[cols_to_explode].map(len).max(axis=1)

    new_records = []

    for idx, row in new_df.iterrows():
        max_len = max_lengths[idx]
        base_row = row.drop(cols_to_explode).tolist()
        
        locations = row['location'] + [''] * (max_len - len(row['location']))
        levels = row['level'] + [''] * (max_len - len(row['level']))
        industries = row['industry'] + [''] * (max_len - len(row['industry']))
        descriptions = row['description'] + [''] * (max_len - len(row['description']))
        
        for i in range(max_len):
            new_row = base_row + [
                locations[i],
                levels[i],
                industries[i],
                descriptions[i]
            ]
            new_records.append(new_row)

    columns = list(new_df.columns.drop(cols_to_explode)) + cols_to_explode
    return pd.DataFrame(new_records, columns=columns)


result_df=explode_data(df)
result_df

,source,job,month,year,job_id,job_title,salary,exp,company_name,location,level,industry,description
0,Jobsgo,Data Analyst,11,2025,1765776,Nhân Viên Phân Tích Dữ Liệu Và Xử Lý Đơn Hàng,15.0,1.0,Công Ty Cổ Phần Tân Phú Việt Nam,Hà Nội,Nhân Viên,Kinh doanh,Python
1,Jobsgo,Data Analyst,11,2025,1765776,Nhân Viên Phân Tích Dữ Liệu Và Xử Lý Đơn Hàng,15.0,1.0,Công Ty Cổ Phần Tân Phú Việt Nam,,Chuyên Viên,,R
2,Jobsgo,Data Analyst,11,2025,1765776,Nhân Viên Phân Tích Dữ Liệu Và Xử Lý Đơn Hàng,15.0,1.0,Công Ty Cổ Phần Tân Phú Việt Nam,,,,Power BI
3,Jobsgo,Data Analyst,11,2025,1757206,Business Analyst (Middle),25.0,3.0,Công Ty Cổ Phần Công Nghệ Phần Mềm Tinh Vân,Hà Nội,Nhân Viên,Kinh Doanh,R
4,Jobsgo,Data Analyst,11,2025,1757206,Business Analyst (Middle),25.0,3.0,Công Ty Cổ Phần Công Nghệ Phần Mềm Tinh Vân,,Chuyên Viên,Công nghệ thông tin,ODI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,Topcv,Business Analyst,11,2025,1680311,Business Analyst,11.5,1.0,CÔNG TY CỔ PHẦN CÔNG NGHỆ VÀ GIẢI PHÁP NMB,Hà Nội,Trưởng nhóm,Khác,R
1636,Topcv,Business Analyst,11,2025,1680311,Business Analyst,11.5,1.0,CÔNG TY CỔ PHẦN CÔNG NGHỆ VÀ GIẢI PHÁP NMB,Cầu Giấy,,,Jira
1637,Topcv,Business Analyst,11,2025,1680311,Business Analyst,11.5,1.0,CÔNG TY CỔ PHẦN CÔNG NGHỆ VÀ GIẢI PHÁP NMB,,,,Balsamiq
1638,Topcv,Business Analyst,11,2025,1680311,Business Analyst,11.5,1.0,CÔNG TY CỔ PHẦN CÔNG NGHỆ VÀ GIẢI PHÁP NMB,,,,Confluence
